In [1]:
using TextAnalysis

┌ Info: Precompiling TextAnalysis [a2db99b7-8b79-58f8-94bf-bbc811eef33d]
└ @ Base loading.jl:1260
┌ Warning: Package WordTokenizers does not have GoogleDrive in its dependencies:
│ - If you have WordTokenizers checked out for development and have
│   added GoogleDrive as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with WordTokenizers
└ Loading GoogleDrive into WordTokenizers from project dependency, future warnings for WordTokenizers are suppressed.
┌ Warning: Package TextAnalysis does not have Requires in its dependencies:
│ - If you have TextAnalysis checked out for development and have
│   added Requires as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with TextAnalysis
└ Loading Requires into TextAnalysis from project dependency, future warnings for TextAnalysis are suppressed.
  ** 

In [2]:
using TextAnalysis.ALBERT

In [3]:
using BSON: @save, @load
@load "/home/iamtejas/Downloads/albert_base_v1.bson.tfbson" config weights vocab
transformer = TextAnalysis.ALBERT.load_pretrainedalbert(config, weights)

3-element Array{Any,1}:
 CompositeEmbedding(tok = Embed(128), segment = Embed(128), pe = PositionEmbedding(128, max_len=512), postprocessor = Positionwise(LayerNorm(128), Dropout(0.1)))
 TextAnalysis.ALBERT.albert_transformer(Dense(128, 768), TextAnalysis.ALBERT.ALGroup(Stack(Transformer(head=12, head_size=64, pwffn_size=3072, size=768)), Dropout(0.1)), 12, 1, 1)
 (pooler = Dense(768, 768, tanh), masklm = (transform = Chain(Dense(768, 128, gelu), LayerNorm(128)), output_bias = Float32[-5.345022, 2.1769698, -7.144285, -9.102521, -8.083536, 0.56541324, 1.2000155, 1.4699979, 1.5557922, 1.9452884  …  -0.6403663, -0.9401073, -1.0888876, -0.9298268, -0.64744073, -0.47156653, -0.81416136, -0.87479985, -0.8785063, -0.5505797]), nextsentence = Chain(Dense(768, 2), logsoftmax))

In [4]:
using WordTokenizers

In [5]:
sample1 = "God is Great! I won a lottery."
sample2 = "If all their conversations in the three months he had been coming to the diner were put together, it was doubtful that they would make a respectable paragraph."
sample3 = "She had the job she had planned for the last three years."
sample = [sample1,sample2,sample3]

3-element Array{String,1}:
 "God is Great! I won a lottery."
 "If all their conversations in the three months he had been coming to the diner were put together, it was doubtful that they would make a respectable paragraph."
 "She had the job she had planned for the last three years."

In [6]:
spm = load(Albert_Version1,"albert_base_v1_30k-clean.vocab")

WordTokenizers.Sentencepiecemodel(["<pad>", "<unk>", "[CLS]", "[SEP]", "[MASK]", "(", ")", "\"", "-", "."  …  "_archivist", "_obverse", "error", "_tyrion", "_addictive", "_veneto", "_colloquial", "agog", "_deficiencies", "_eloquent"], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  -13.5298, -13.5298, -13.5298, -13.5299, -13.5299, -13.53, -13.5313, -13.5318, -13.5323, -13.5323])

In [7]:
s1 = ids_from_tokens(spm, tokenizer(spm,sample[1]))
s2 = ids_from_tokens(spm, tokenizer(spm,sample[2]))
s3 = ids_from_tokens(spm, tokenizer(spm,sample[3]))
E = Flux.batchseq([s1,s2,s3],1)
E = Flux.stack(E,1)

32×3 Array{Int64,2}:
   14     14    14
    2      2     2
 5649    411   439
   26     66    42
   14     67    15
    2  13528  1206
  100     20    40
  722     15    42
  188    133  2036
   14    819    27
    2     25    15
  231     42   237
   22     75   133
    ⋮         
    1     16     1
    1     33     1
    1     24     1
    1  22569     1
    1     31     1
    1     60     1
    1     84     1
    1    234     1
    1     22     1
    1  22740     1
    1  20600     1
    1     10     1

In [8]:
seg_indices = ones(Int, size(E)...)

32×3 Array{Int64,2}:
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 ⋮     
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1
 1  1  1

In [9]:
embedding = transformer[1]
emb = embedding(tok=E, segment=seg_indices)

128×32×3 Array{Float32,3}:
[:, :, 1] =
  0.215637     0.974126    0.923233   …  -0.953364   -1.23193    -1.45834
  0.382527    -0.440211   -0.493132      -0.450212   -0.497128   -0.664383
 -1.06308     -0.596149    1.38881       -0.893396   -0.817059   -0.753805
  0.0192553   -0.606222   -0.0422194      0.213562    0.366818    0.655723
 -1.73208     -0.217243    0.329404       0.720658    0.467273    0.371431
  1.44428      0.352459   -0.307714   …   0.729       0.681432    0.574053
 -2.87975      1.02795    -1.55634       -0.0406037  -0.331129   -0.634261
  0.888469     2.01966     0.139051      -1.54702    -1.5215     -1.46764
  0.423823     0.10119    -1.7348        -1.40969    -0.998497   -0.439296
  0.519439     0.50028     1.78553        1.24271     0.801729    0.306854
 -2.73993     -2.15839    -0.950032   …  -0.726077   -1.00341    -1.15917
 -0.739749    -0.58749     0.697434       2.10209     1.90288     1.72213
 -2.06217     -1.16219     1.14314        1.72633     1.4998     

In [10]:
contextualised_embedding = transformer[2](emb)

768×32×3 Array{Float32,3}:
[:, :, 1] =
  0.304567    0.405941    0.326642   …   0.527699    0.53062     0.535387
  0.765856    1.05468     0.581975       0.902422    0.923356    0.910047
  0.760675    0.440686    1.81601        0.665885    0.615429    0.596886
 -0.0437517  -0.464514   -0.133041      -0.0119426   0.0575484   0.0867782
  0.438816   -0.0251722   0.686429       0.80559     0.805196    0.780889
  0.0296612  -0.685908    0.21051    …  -0.175856   -0.189034   -0.196193
  0.646657    1.67699     0.259575       0.684573    0.667937    0.646987
  0.267509    1.71205     0.686641       0.999966    1.0456      1.06525
 -0.0601687  -0.354132   -0.886672      -0.617012   -0.610717   -0.594557
 -0.364813   -0.159958   -0.996861      -0.187835   -0.205887   -0.215373
  0.988467    1.11282     1.00583    …   0.78229     0.908326    0.959077
  0.0731863  -0.432719    0.150405       0.585053    0.49542     0.441913
  0.15807    -0.490142   -0.405144       0.355488    0.33691     0.322694

In [11]:
cls = transformer[3]

(pooler = Dense(768, 768, tanh), masklm = (transform = Chain(Dense(768, 128, gelu), LayerNorm(128)), output_bias = Float32[-5.345022, 2.1769698, -7.144285, -9.102521, -8.083536, 0.56541324, 1.2000155, 1.4699979, 1.5557922, 1.9452884  …  -0.6403663, -0.9401073, -1.0888876, -0.9298268, -0.64744073, -0.47156653, -0.81416136, -0.87479985, -0.8785063, -0.5505797]), nextsentence = Chain(Dense(768, 2), logsoftmax))

In [12]:
using Transformers.Basic:@toNd
cls.pooler(contextualised_embedding[:,1,:]) #mostly pooler layer is applied on [cls] token

768×3 Array{Float32,2}:
 -0.164622   -0.244687   -0.337835
 -0.109313   -0.238238    0.0731121
 -0.922526   -0.99792    -0.46698
 -0.403019   -0.863416    0.702999
  0.439029    0.97862    -0.107664
 -0.852264   -0.900703   -0.614105
  0.364179   -0.373978    0.0256299
 -0.508897   -0.47372     0.164483
  0.536143    0.0399534   0.775943
  0.970573    0.768918    0.205684
  0.999793    0.998327    0.999979
  0.791798   -0.33849     0.438395
 -0.503646    0.227776    0.245557
  ⋮                      
  0.757302    0.637661    0.859677
 -0.819503   -0.998739   -0.814534
 -0.505278   -0.654982   -0.670874
 -0.994383    0.0932321  -0.308428
 -0.6843     -0.984353   -0.602701
  0.751023    0.684121    0.818537
 -0.997563   -0.990052   -0.98816
 -0.405217   -0.750618   -0.819593
  0.53443     0.589601    0.67484
  0.973679    0.97328     0.951646
 -0.880809   -0.918532    0.0197378
 -0.0609629  -0.814388   -0.487664

In [21]:
s1 =  tokenizer(spm,sample[1])
s2 =  tokenizer(spm,sample[2])
s3 =  tokenizer(spm,sample[3])
s =[s1,s2,s3]

3-element Array{Array{String,1},1}:
 ["_", "G", "od", "_is", "_", "G", "re", "at", "!", "_", "I", "_won", "_a", "_lottery", "."]
 ["_", "I", "f", "_all", "_their", "_conversations", "_in", "_the", "_three", "_months"  …  "_was", "_doubtful", "_that", "_they", "_would", "_make", "_a", "_respectable", "_paragraph", "."]
 ["_", "S", "he", "_had", "_the", "_job", "_she", "_had", "_planned", "_for", "_the", "_last", "_three", "_years", "."]

In [35]:
using Transformers
masks = Transformers.Basic.getmask(s)

1×32×3 Array{Float32,3}:
[:, :, 1] =
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2] =
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0

[:, :, 3] =
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [23]:
contextualised_embedding = transformer[2](emb, masks)

768×32×3 Array{Float32,3}:
[:, :, 1] =
 -0.428328   -0.0777661  -0.221679    …   0.0   0.0   0.0   0.0   0.0   0.0
 -0.569262    0.571153   -0.281473        0.0   0.0   0.0   0.0   0.0   0.0
  0.0193166   0.0451773   2.74292         0.0   0.0   0.0   0.0   0.0   0.0
 -0.978615   -1.16123    -0.574968       -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
  0.201378   -0.735267    0.0164461       0.0   0.0   0.0   0.0   0.0   0.0
 -0.322908   -0.379362   -0.813322    …  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
  0.660336    2.02064     0.649449        0.0   0.0   0.0   0.0   0.0   0.0
 -0.91708     0.392511   -0.667269       -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.254425   -0.426472   -1.18446        -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
  0.617348    1.62654     0.141215        0.0   0.0   0.0   0.0   0.0   0.0
  1.55256     2.01011     1.37082     …   0.0   0.0   0.0   0.0   0.0   0.0
  0.0662555  -0.670731    0.121412       -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
  0.127238   -0.539187   -0.00406911     -0.0  -0